In [1]:
import json
import pandas as pd
import sqlite3
import Coinex_API_Class as Coinex

%reload_ext autoreload
%autoreload 2

try :
    with open('config.json', 'r') as f:
        data = json.load(f)
        api_key = data['api_key']
        api_secret = data['api_secret']
        db_name = data['db_name']
        num_candles = data['num_candle']
        loss_limit = data['loss_limit']
        client_id = data['client_id']
except Exception as e:
    print(f"Error: {e}")

conn = sqlite3.connect("crypto.db")
coinex = Coinex.Coinex_API(api_key, api_secret, connection=conn , client_id=client_id)

In [ ]:
import threading
import time

def print_numbers():
    for i in range(5):
        print(f"عدد {i}")
        time.sleep(1)

def print_letters():
    for letter in ['A', 'B', 'C', 'D', 'E']:
        print(f"حرف {letter}")
        time.sleep(1.5)

# ایجاد دو Thread برای اجرای وظایف مختلف
thread1 = threading.Thread(target=print_numbers)
thread2 = threading.Thread(target=print_letters)

# شروع اجرای Threads
thread1.start()
thread2.start()

# منتظر ماندن تا اتمام اجرای Threads
thread1.join()
thread2.join()

print("پایان اجرای همروندی")

In [6]:
coinex.get_spot_price_ticker("BTCUSDT")

[{'close': '68169.99',
  'high': '68451.65',
  'last': '68169.99',
  'low': '68035.58',
  'market': 'BTCUSDT',
  'open': '68445.77',
  'period': 86400,
  'value': '21572344.9779615622',
  'volume': '316.14646532',
  'volume_buy': '2.74274679',
  'volume_sell': '5.57223814'}]

In [ ]:
cursor = conn.cursor()
query = "SELECT * FROM transactions"  
df = pd.read_sql_query(query, conn)
df_b = df[df['sold']== 'USDT'].copy()
df2=pd.DataFrame()
i=0
for index, row in df_b.iterrows():
    df2.loc[i,'Time_buy']= row['ltime']
    df2.loc[i,'ccy'] = row['buy']
    df2.loc[i,'pure_amount']= row['balance']
    df2.loc[i,'pay_USDT']= row['pay']
    df2.loc[i,'Time_sold']= None
    df2.loc[i,'recieve']= None
    df2.loc[i,'proft']= None 
    i +=1

df2.to_sql('profit', conn, if_exists='replace', index=False)

In [ ]:
# -*- coding: utf-8 -*-

import asyncio
import websockets
import json
import time
import hashlib
import gzip
import hmac

WS_URL = "wss://socket.coinex.com/v2/spot"  # Change "spot" to "futures" when interacting with WS ports
access_id = "47D7C4B286224298BB3D88A9D7161A45"  # Replace with your access id
secret_key = "F0F2D4156F6892ED2F03576FEE74CFA658C5F8271DCE102D"  # Replace with your secret key


async def ping(conn):
    param = {"method": "server.ping", "params": {}, "id": 1}
    while True:
        await conn.send(json.dumps(param))
        await asyncio.sleep(3)


async def auth(conn):
    timestamp = int(time.time() * 1000)

    # Generate your signature string
    prepared_str = f"{timestamp}"
    signed_str = (
        hmac.new(
            bytes(secret_key, "latin-1"),
            msg=bytes(prepared_str, "latin-1"),
            digestmod=hashlib.sha256,
        )
        .hexdigest()
        .lower()
    )

    param = {
        "method": "server.sign",
        "params": {
            "access_id": access_id,
            "signed_str": signed_str,
            "timestamp": timestamp,
        },
        "id": 1,
    }
    await conn.send(json.dumps(param))
    res = await conn.recv()
    res = gzip.decompress(res)
    print("Authentication Result: ", json.loads(res))


async def subscribe_depth(conn):
    param = {
        "method": "depth.subscribe",
        "params": {"market_list": [["BTCUSDT", 5, "0", True]]},
        "id": 1,
    }
    await conn.send(json.dumps(param))
    res = await conn.recv()
    res = gzip.decompress(res)
    print(json.loads(res))


async def subscribe_asset(conn):
    param = {"method": "balance.subscribe", "params": {"ccy_list": ["USDT"]}, "id": 1}
    await conn.send(json.dumps(param))
    res = await conn.recv()
    res = gzip.decompress(res)
    print(json.loads(res))


async def main():
    try:
        # Note: Must close websockets ping feature before creating a new ping task, set ping_interval to None
        async with websockets.connect(
            uri=WS_URL, compression=None, ping_interval=None
        ) as conn:
            await auth(conn)
            await subscribe_depth(conn)
            await subscribe_asset(conn)

            asyncio.create_task(ping(conn))

            while True:
                res = await conn.recv()
                res = gzip.decompress(res)
                res = json.loads(res)
                print(res)
    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    asyncio.run(main())
